#300 was trained on 100 epochs

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import numpy as np
import tensorflow as tf
path = r'..\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 300
with open(path + "base2021_300", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

261068


In [4]:
print(type(word_dict))

<class 'dict'>


In [5]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    return index

In [6]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [ ]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('smaller',7))#this should produce one, for the same vectors
print(find_nearest_k('computer',7))

## 3cosAdd 

In [7]:

def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# 3cosAdd normalised befor arithmetic

In [8]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

## 3cos Mult

In [9]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        print(a in id_dict)
        print(a_star in id_dict)
        print(b in id_dict)
        print(b_star in id_dict)
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# Test all

In [10]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        lines = file.readlines()
        for line_id,line in enumerate(lines):
            if (line_id % 100 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            #predicted_3cos_add      = three_cos_predict_np(a,a_star,b,b_star)
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            #predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            #print(a,a_star,b,b_star)
            #print(predicted_3cos_add,predicted_3cos_add_norm,predicted_3cos_mult)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_sem_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    count_sem_sucess[1] += 1
                else:
                    errors.append(predicted_3cos_add_norm)
                #if  predicted_3cos_mult    == b_star:
                #    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_syn_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    count_syn_sucess[1] += 1
                else:
                    errors.append(predicted_3cos_add_norm)
                #if  predicted_3cos_mult    == b_star:
                #    count_syn_sucess[2] += 1
    
print(count_sem_questions)
print(count_syn_questions)
print(count_sem_sucess)
print(count_syn_sucess)
print(errors)

capital-common-countries.txt
0
100
200
300
400
500
capital-world.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
city-in-state.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
currency.txt
0
100
200
300
400
500
600
700
800
family.txt
0
100
200
300
400
500
gram1-adjective-to-adverb.txt
0
100
200
300
400
500
600
700
800
900
gram2-opposite.txt
0
100
200
300
400
500
600
700
800
gram3-comparative.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
gram4-superlative.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
gram5-present-participle.txt
0
100
200
300
400
500
600
700
800
900
1000
gram6-nationality-adjective.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
gram7-past-tense.txt
0
100
200
300
400
500
600
700
800
900


In [13]:
import winsound
winsound.Beep(440, 500)

In [15]:
import operator
dict = {}
for error in errors:
    if error in dict:
        dict[error]+=1
    else:
        dict[error]= 1
sorted_d = sorted(dict.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_d[0:100])

[(None, 124), ('kansas', 98), ('fast', 93), ('chicago', 70), ('multibillion', 66), ('york', 60), ('washington', 49), ('aleck', 45), ('jumps', 44), ('colorado', 43), ('arizona', 43), ('goldstar', 42), ('higher', 42), ('vanishes', 41), ('slovak', 40), ('really', 40), ('much', 40), ('stating', 39), ('coolest', 38), ('belarusian', 38), ('know', 37), ('colour', 36), ('feed', 36), ('missouri', 35), ('guava', 35), ('increased', 34), ('codes', 32), ('canada', 31), ('increase', 31), ('oregon', 30), ('illinois', 30), ('argentine', 30), ('prickly', 30), ('fortunately', 29), ('pool', 29), ('dance', 29), ('ohio', 28), ('angeles', 28), ('uncommon', 28), ('flies', 28), ('predict', 28), ('walk', 28), ('rodders', 27), ('blacula', 27), ('konga', 27), ('enhance', 26), ('cleveland', 25), ('leviev', 25), ('screams', 25), ('cantaloupe', 25), ('garlic', 25), ('scott', 24), ('incredible', 24), ('baku', 23), ('improve', 23), ('generate', 23), ('peppa', 23), ('predicting', 22), ('hoyas', 21), ('michigan', 21), 

In [3]:
(4537 + 2961) / (8869 + 10675)

0.3836471551371265

In [4]:
(4594 + 3144) / (8869 + 10675)

0.39592713876381497

In [12]:
(4526 + 5252) / (8869 + 10675)

0.5003069995906673

In [11]:
(3943 + 5218) / (8869 + 10675)

0.46873720835038885

In [13]:
(4686 + 5445) / (8869 + 10675)

0.5183688088415882

In [17]:
(6112 + 6121) / (8869 + 10675)

0.6259209987720017

In [16]:
(6112 + 6121 + 80) / (8869 + 10675)

0.6300143266475645